In [1]:
!pip install python-pptx python-docx
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [2]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from pptx import Presentation
from docx import Document
import io

# Authenticate and set up Drive service
auth.authenticate_user()
service = build('drive', 'v3')

# Specify the folder ID for the folder named "tes"
folder_id = '1v8XmlQE9tqfx74-5LUJrotVWxyZIPAYx'  # Replace with your actual folder ID

# Build the query to search for Google Docs, PDF, PPTX, and DOCX files in the specified folder
query = f"'{folder_id}' in parents and (mimeType='application/vnd.google-apps.document' or mimeType='application/pdf' or mimeType='application/vnd.openxmlformats-officedocument.presentationml.presentation' or mimeType='application/vnd.openxmlformats-officedocument.wordprocessingml.document')"

# List the files in the specific folder
results = service.files().list(q=query, pageSize=10, fields="nextPageToken, files(id, name, mimeType)").execute()
items = results.get('files', [])

if not items:
    print('No files found in the folder.')
else:
    print('Files in folder:')
    for item in items:
        print(f"{item['name']} ({item['id']}) - {item['mimeType']}")

# Function to download and extract text from files
def download_and_extract_text(file_id, mime_type):
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()

    fh.seek(0)

    # Extract text based on MIME type
    if mime_type == 'application/vnd.google-apps.document':
        return fh.getvalue().decode('utf-8')

    elif mime_type == 'application/pdf':
        from PyPDF2 import PdfReader
        pdf_reader = PdfReader(fh)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() or ""
        return text

    elif mime_type == 'application/vnd.openxmlformats-officedocument.presentationml.presentation':  # PPTX
        pptx_file = io.BytesIO(fh.read())
        presentation = Presentation(pptx_file)
        text = ""
        for slide in presentation.slides:
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    text += shape.text + "\n"
        return text

    elif mime_type == 'application/vnd.openxmlformats-officedocument.wordprocessingml.document':  # DOCX
        docx_file = io.BytesIO(fh.read())
        document = Document(docx_file)
        text = ""
        for paragraph in document.paragraphs:
            text += paragraph.text + "\n"
        return text

    return None

# Extract text from files in the folder
documents_text = []
for item in items:
    file_id = item['id']
    mime_type = item['mimeType']
    text = download_and_extract_text(file_id, mime_type)
    documents_text.append((item['name'], text))

    print(f"Text from {item['name']}:\n{text[:100]}...\n")

Files in folder:
pembinaan jf sandiman dan MI 2022 final.pptx (15LCB3Qf6ejNofqehPxZcdx63vxxOiora) - application/vnd.openxmlformats-officedocument.presentationml.presentation
Notulen Enhancing Cybersecurity Awareness in Facing Cyber Threats.docx (1tHTa1Y2ZcJMluhqEyIeqyvOGHGxSo4LK) - application/vnd.openxmlformats-officedocument.wordprocessingml.document
Paparan Keamanan Informasi (1).pdf (1mGQ7Dhuapr35mzhXZbhWV8DyKfy06VJv) - application/pdf
Text from pembinaan jf sandiman dan MI 2022 final.pptx:
”PEMBINAAN JF SANDIMAN DAN 
MANGGALA INFORMATIKA” 




















VISI INDONESIA 2045:
BERDAULA...

Text from Notulen Enhancing Cybersecurity Awareness in Facing Cyber Threats.docx:
NOTULEN ACARA Enhancing Cybersecurity Awareness in Facing Cyber Threats

Hari 	: Kamis
Tanggal 	: 24...

Text from Paparan Keamanan Informasi (1).pdf:
Sosialisasi Keamanan
Informasi
Semarang, 21-24 Oktober 2024Dinas Komunikasi Informatika
Statistik da...



In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [4]:
from langchain.schema.document import Document

def load_documents():
    documents = []
    for name, text in documents_text:
      documents.append(Document(page_content=text, metadata={"source": name}))
    return documents

In [5]:
documents = load_documents()
chunks = split_documents(documents)
print(chunks[0])

page_content='”PEMBINAAN JF SANDIMAN DAN 
MANGGALA INFORMATIKA” 




















VISI INDONESIA 2045:
BERDAULAT,  MAJU, ADIL DAN  MAKMUR
Revolusi Industri 4.0
Manusia Indonesia yang unggul,
berbudaya, serta menguasai ilmu pengetahuan dan teknologi
SDM  ASN  yang  profesional, bersih, kompeten,  netral dan  berintegritas sangat berperan dalam menentukan efektivitas pemerintah untuk mewujudkan visi pembangunan
Ekonomi yang maju dan
berkelanjutan
Pembangunan yang merata dan
inklusif
Negara yang demokratis, kuat,
dan bersih
Future of Government
(digital & citizen centric new 
government model)

Menuju Indonesia 2045
Lingkungan Strategis & Kebutuhan ASN berkualitas' metadata={'source': 'pembinaan jf sandiman dan MI 2022 final.pptx'}


In [6]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13269    0 13269    0     0  51843      0 --:--:-- --:--:-- --:--:-- 52035
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [7]:
import subprocess
import time

# Start ollama as a backrgound process
command = "nohup ollama serve&"

# Use subprocess.Popen to start the process in the background
process = subprocess.Popen(command,
                            shell=True,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE)
print("Process ID:", process.pid)
# Let's use fly.io resources
#!OLLAMA_HOST=https://ollama-demo.fly.dev:443
time.sleep(5)  # Makes Python wait for 5 seconds

Process ID: 1230


In [8]:
!ollama -v

ollama version is 0.5.7


In [9]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14


In [10]:
from langchain_community.embeddings.ollama import OllamaEmbeddings
def get_embedding_function():
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    return embeddings

In [11]:
from langchain.vectorstores.chroma import Chroma
CHROMA_PATH='chroma'
def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=get_embedding_function()
    )
    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")

def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks


In [12]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.8/70.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.6 MB/s eta 0:00:0

In [13]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 970aa74c0a90...   0% ▕▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕▏ 1.0 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...   7% ▕▏  17 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  13% ▕▏  35 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  17% ▕▏  47 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  25% ▕▏  69 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  31% ▕▏  85 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  34% ▕▏  94 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  41% ▕▏ 111 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  48% ▕▏ 130 MB/

In [14]:
# Create (or update) the data store.
documents = load_documents()
chunks = split_documents(documents)
add_to_chroma(chunks)

<ipython-input-10-35cf5b50dfff>:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text")
<ipython-input-11-acaf1b5d3dc1>:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


Number of existing documents in DB: 0
👉 Adding new documents: 42


<ipython-input-11-acaf1b5d3dc1>:26: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [15]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""
def query_rag(query_text: str):
    # Prepare the DB.
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    model = Ollama(model="llama2")
    response_text = model.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text


In [16]:
!ollama pull llama2

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏ 2.2 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏ 5.9 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   1% ▕▏  25 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   1% ▕▏  39 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   1% ▕▏  57 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   2% ▕▏  89 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   3% ▕▏ 108 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   4% ▕▏ 134 MB/3.8 GB                  pulling manife

In [19]:
query_text = "tips agar terhindar dari serangan slot gacor"
query_rag(query_text)

Response: Based on the provided context, here are some tips to avoid being attacked by a cyber attack:

1. Use strong and unique passwords: Create a password that is difficult to guess and contains a mix of uppercase and lowercase letters, numbers, and special characters. Avoid using common words or phrases that can be easily found in a dictionary.
2. Keep your software up-to-date: Ensure that your operating system, security patches, plugins, and other software are updated regularly to fix vulnerabilities and protect against the latest threats.
3. Implement least privilege principle: Limit user access to only what is necessary for their work, this can help prevent attacks from exploiting privilege escalation vulnerabilities.
4. Practice input sanitization: Validate and clean user input to prevent attackers from exploiting security vulnerabilities through malicious inputs.
5. Conduct regular security assessments: Perform security assessments regularly to identify and address potential v

"Based on the provided context, here are some tips to avoid being attacked by a cyber attack:\n\n1. Use strong and unique passwords: Create a password that is difficult to guess and contains a mix of uppercase and lowercase letters, numbers, and special characters. Avoid using common words or phrases that can be easily found in a dictionary.\n2. Keep your software up-to-date: Ensure that your operating system, security patches, plugins, and other software are updated regularly to fix vulnerabilities and protect against the latest threats.\n3. Implement least privilege principle: Limit user access to only what is necessary for their work, this can help prevent attacks from exploiting privilege escalation vulnerabilities.\n4. Practice input sanitization: Validate and clean user input to prevent attackers from exploiting security vulnerabilities through malicious inputs.\n5. Conduct regular security assessments: Perform security assessments regularly to identify and address potential vuln